# Autoencoders

In [ ]:
# Author: Hamaad Musharaf Shah.
# The following references were used.
# https://keras.io
# https://blog.keras.io/building-autoencoders-in-keras.html
# https://stackoverflow.com/questions/42177658/how-to-switch-backend-with-keras-from-tensorflow-to-theano
# https://towardsdatascience.com/learning-rate-schedules-and-adaptive-learning-rate-methods-for-deep-learning-2c8f433990d1
# http://scikit-learn.org/stable/
# Book: Ian Goodfellow, Yoshua Bengio and Aaron Courville, "Deep Learning" - http://www.deeplearningbook.org
# Book: Aurelien Geron, "Hands-On Machine Learning with Scikit-Learn & Tensorflow" - https://www.amazon.co.uk/Hands-Machine-Learning-Scikit-Learn-TensorFlow/dp/1491962291

import os
import math
import sys
import importlib

import numpy as np

import pandas as pd

from sklearn import linear_model
from sklearn.datasets import fetch_mldata
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, LabelBinarizer
from sklearn.metrics import roc_auc_score

from scipy.stats import norm

import keras
from keras import backend as bkend
from keras.datasets import cifar10

from autoencoders_keras.get_session import get_session
import keras.backend.tensorflow_backend as KTF
KTF.set_session(get_session(gpu_fraction=0.25, allow_soft_placement=True))

import tensorflow as tf
from tensorflow.python.client import device_lib

from plotnine import *

import matplotlib.pyplot as plt

from autoencoders_keras.vanilla_autoencoder import VanillaAutoencoder
from autoencoders_keras.convolutional_autoencoder import ConvolutionalAutoencoder
from autoencoders_keras.convolutional2D_autoencoder import Convolutional2DAutoencoder
from autoencoders_keras.seq2seq_autoencoder import Seq2SeqAutoencoder
from autoencoders_keras.variational_autoencoder import VariationalAutoencoder

%matplotlib inline

os.environ["KERAS_BACKEND"] = "tensorflow"
importlib.reload(bkend)

print(device_lib.list_local_devices())

data_set = "CIFAR"

In [ ]:
if data_set == "MNIST":
    mnist = fetch_mldata("MNIST original")
    X, y = mnist["data"], mnist["target"]
    X = X.astype("float32")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=666)
    X_train /= 255.0
    X_test /= 255.0
elif data_set == "CIFAR":
    cifar = cifar10.load_data()
    (X_train, y_train), (X_test, y_test) = cifar
    y_train = y_train.ravel()
    y_test = y_test.ravel()
    X_train = X_train.astype("float32")
    X_test = X_test.astype("float32")
    X_train /= 255.0
    X_test /= 255.0

In [ ]:
scaler_classifier = MinMaxScaler(feature_range=(0.0, 1.0))
logistic = linear_model.LogisticRegression(random_state=666)
lb = LabelBinarizer()
lb = lb.fit(y_train.reshape(y_train.shape[0], 1))

In [ ]:
pipe_base = Pipeline(steps=[("scaler_classifier", scaler_classifier),
                            ("classifier", logistic)])
pipe_base = pipe_base.fit(X_train, y_train)

metric_base = roc_auc_score(lb.transform(y_test.reshape(y_test.shape[0], 1)), 
                            pipe_base.predict_proba(X_test), 
                            average="weighted")
print(metric_base)

In [ ]:
with tf.device("/gpu:0"):
    autoencoder = VanillaAutoencoder(n_feat=X_train.shape[1],
                                     n_epoch=5,
                                     batch_size=100,
                                     encoder_layers=3,
                                     decoder_layers=3,
                                     n_hidden_units=int(X_train.shape[1] / 2),
                                     encoding_dim=int(X_train.shape[1] / 2),
                                     denoising=None)

    print(autoencoder.autoencoder.summary())

    pipe_autoencoder = Pipeline(steps=[("autoencoder", autoencoder),
                                       ("scaler_classifier", scaler_classifier),
                                       ("classifier", logistic)])

    pipe_autoencoder = pipe_autoencoder.fit(X_train, y_train)

    metric_autoencoder = roc_auc_score(lb.transform(y_test.reshape(y_test.shape[0], 1)), 
                                       pipe_autoencoder.predict_proba(X_test), 
                                       average="weighted")
    print(metric_autoencoder)

In [ ]:
with tf.device("/gpu:0"):
    noise = 0.10 * np.reshape(np.random.uniform(low=0.0, 
                                                high=1.0, 
                                                size=X_train.shape[0] * X_train.shape[1]), 
                              [X_train.shape[0], X_train.shape[1]])

    denoising_autoencoder = VanillaAutoencoder(n_feat=X_train.shape[1],
                                               n_epoch=5,
                                               batch_size=100,
                                               encoder_layers=3,
                                               decoder_layers=3,
                                               n_hidden_units=int(X_train.shape[1] / 2),
                                               encoding_dim=int(X_train.shape[1] / 2),
                                               denoising=noise)

    print(denoising_autoencoder.autoencoder.summary())

    pipe_denoising_autoencoder = Pipeline(steps=[("autoencoder", denoising_autoencoder),
                                                 ("scaler_classifier", scaler_classifier),
                                                 ("classifier", logistic)])

    pipe_denoising_autoencoder = pipe_denoising_autoencoder.fit(X_train, y_train)

    metric_denoising_autoencoder = roc_auc_score(lb.transform(y_test.reshape(y_test.shape[0], 1)), 
                                                 pipe_denoising_autoencoder.predict_proba(X_test), 
                                                 average="weighted")
    print(metric_denoising_autoencoder)

In [ ]:
with tf.device("/gpu:0"):
    convolutional_autoencoder = ConvolutionalAutoencoder(input_shape=(int(math.pow(X_train.shape[1], 0.5)), int(math.pow(X_train.shape[1], 0.5))),
                                                         n_epoch=5,
                                                         batch_size=100,
                                                         encoder_layers=3,
                                                         decoder_layers=3,
                                                         filters=100,
                                                         kernel_size=5,
                                                         strides=1,
                                                         pool_size=4,
                                                         encoding_dim=14,
                                                         denoising=None)

    print(convolutional_autoencoder.autoencoder.summary())

    pipe_convolutional_autoencoder = Pipeline(steps=[("autoencoder", convolutional_autoencoder),
                                                     ("scaler_classifier", scaler_classifier),
                                                     ("classifier", logistic)])

    pipe_convolutional_autoencoder = pipe_convolutional_autoencoder.fit(np.reshape(X_train, [X_train.shape[0], int(math.pow(X_train.shape[1], 0.5)), int(math.pow(X_train.shape[1], 0.5))]), 
                                                                        y_train)

    metric_convolutional_autoencoder = roc_auc_score(lb.transform(y_test.reshape(y_test.shape[0], 1)), 
                                                     pipe_convolutional_autoencoder.predict_proba(np.reshape(X_test, [X_test.shape[0], int(math.pow(X_train.shape[1], 0.5)), int(math.pow(X_train.shape[1], 0.5))])),
                                                     average="weighted")
    print(metric_convolutional_autoencoder)

In [ ]:
with tf.device("/gpu:0"):
    seq2seq_autoencoder = Seq2SeqAutoencoder(input_shape=(int(math.pow(X_train.shape[1], 0.5)), int(math.pow(X_train.shape[1], 0.5))),
                                             n_epoch=5,
                                             batch_size=100,
                                             encoder_layers=3,
                                             decoder_layers=3,
                                             n_hidden_units=50,
                                             encoding_dim=14,
                                             stateful=False,
                                             denoising=None)

    print(seq2seq_autoencoder.autoencoder.summary())

    pipe_seq2seq_autoencoder = Pipeline(steps=[("autoencoder", seq2seq_autoencoder),
                                               ("scaler_classifier", scaler_classifier),
                                               ("classifier", logistic)])

    pipe_seq2seq_autoencoder = pipe_seq2seq_autoencoder.fit(np.reshape(X_train, [X_train.shape[0], int(math.pow(X_train.shape[1], 0.5)), int(math.pow(X_train.shape[1], 0.5))]),
                                                            y_train)

    metric_seq2seq_autoencoder = roc_auc_score(lb.transform(y_test.reshape(y_test.shape[0], 1)), 
                                               pipe_seq2seq_autoencoder.predict_proba(np.reshape(X_test, [X_test.shape[0], int(math.pow(X_train.shape[1], 0.5)), int(math.pow(X_train.shape[1], 0.5))])),
                                               average="weighted")
    print(metric_seq2seq_autoencoder)

In [ ]:
with tf.device("/gpu:0"):
    encoding_dim = 2

    variational_autoencoder = VariationalAutoencoder(n_feat=X_train.shape[1],
                                                     n_epoch=5,
                                                     batch_size=100,
                                                     encoder_layers=3,
                                                     decoder_layers=3,
                                                     n_hidden_units=int(X_train.shape[1] / 2),
                                                     encoding_dim=encoding_dim,
                                                     denoising=None)

    print(variational_autoencoder.autoencoder.summary())

    pipe_variational_autoencoder = Pipeline(steps=[("autoencoder", variational_autoencoder),
                                                   ("scaler_classifier", scaler_classifier),
                                                   ("classifier", logistic)])

    pipe_variational_autoencoder = pipe_variational_autoencoder.fit(X_train, y_train)

    metric_variational_autoencoder = roc_auc_score(lb.transform(y_test.reshape(y_test.shape[0], 1)), 
                                                   pipe_variational_autoencoder.predict_proba(X_test), 
                                                   average="weighted")
    print(metric_variational_autoencoder)

    if encoding_dim == 2:
        test_encoded_df = pd.DataFrame(pipe_variational_autoencoder.named_steps["autoencoder"].encoder.predict(X_test))
        test_encoded_df["Target"] = y_test
        test_encoded_df.columns.values[0:2] = ["Encoding_1", "Encoding_2"]

        scaler_plot = MinMaxScaler(feature_range=(0.25, 0.75))
        scaler_plot = scaler_plot.fit(test_encoded_df[["Encoding_1", "Encoding_2"]])
        test_encoded_df[["Encoding_1", "Encoding_2"]] = scaler_plot.transform(test_encoded_df[["Encoding_1", "Encoding_2"]])

        cluster_plot = ggplot(test_encoded_df) + \
        geom_point(aes(x="Encoding_1", 
                       y="Encoding_2", 
                       fill="factor(Target)"),
                   size=1,
                   color = "black") + \
        xlab("Encoding dimension 1") + \
        ylab("Encoding dimension 2") + \
        ggtitle("Variational autoencoder with 2-dimensional encoding") + \
        theme_matplotlib()
        print(cluster_plot)

        n = 15
        digit_size = 28
        figure = np.zeros((digit_size * n, digit_size * n))
        grid_x = norm.ppf(np.linspace(0.05, 0.95, n))
        grid_y = norm.ppf(np.linspace(0.05, 0.95, n))

        for i, xi in enumerate(grid_x):
            for j, yi in enumerate(grid_y):
                z_sample = np.array([[xi, yi]])
                x_decoded = pipe_variational_autoencoder.named_steps["autoencoder"].generator.predict(z_sample)
                digit = x_decoded[0].reshape(digit_size, digit_size)
                figure[i * digit_size: (i + 1) * digit_size, j * digit_size: (j + 1) * digit_size] = digit

        plt.figure(figsize=(10, 10))
        plt.imshow(figure, cmap="Greys_r")
        plt.title("Variational autoencoder with 2-dimensional encoding\nGenerating new images")
        plt.xlabel("")
        plt.ylabel("")
        plt.show()

In [ ]:
with tf.device("/gpu:0"):
    convolutional2D_autoencoder = Convolutional2DAutoencoder(input_shape=(int(math.pow(X_train.shape[1], 0.5)), int(math.pow(X_train.shape[1], 0.5)), 1),
                                                             n_epoch=5,
                                                             batch_size=100,
                                                             encoder_layers=3,
                                                             decoder_layers=3,
                                                             filters=100,
                                                             kernel_size=5,
                                                             strides=1,
                                                             pool_size=4,
                                                             encoding_dim=14,
                                                             denoising=None)

    print(convolutional2D_autoencoder.autoencoder.summary())

    pipe_convolutional2D_autoencoder = Pipeline(steps=[("autoencoder", convolutional2D_autoencoder),
                                                       ("scaler_classifier", scaler_classifier),
                                                       ("classifier", logistic)])

    pipe_convolutional2D_autoencoder = pipe_convolutional2D_autoencoder.fit(np.reshape(X_train, [X_train.shape[0], int(math.pow(X_train.shape[1], 0.5)), int(math.pow(X_train.shape[1], 0.5)), 1]),
                                                                            y_train)

    metric_convolutional2D_autoencoder = roc_auc_score(lb.transform(y_test.reshape(y_test.shape[0], 1)), 
                                                       pipe_convolutional2D_autoencoder.predict_proba(np.reshape(X_test, [X_test.shape[0], int(math.pow(X_train.shape[1], 0.5)), int(math.pow(X_train.shape[1], 0.5)), 1])),
                                                       average="weighted")
    print(metric_convolutional2D_autoencoder)

In [ ]:
with tf.device("/gpu:0"):
    convolutional2D_autoencoder = Convolutional2DAutoencoder(input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3]),
                                                             n_epoch=5,
                                                             batch_size=100,
                                                             encoder_layers=3,
                                                             decoder_layers=3,
                                                             filters=50,
                                                             kernel_size=5,
                                                             strides=1,
                                                             pool_size=4,
                                                             encoding_dim=16,
                                                             denoising=None)

    print(convolutional2D_autoencoder.autoencoder.summary())

    pipe_convolutional2D_autoencoder = Pipeline(steps=[("autoencoder", convolutional2D_autoencoder),
                                                       ("scaler_classifier", scaler_classifier),
                                                       ("classifier", logistic)])

    pipe_convolutional2D_autoencoder = pipe_convolutional2D_autoencoder.fit(X_train, y_train)

    metric_convolutional2D_autoencoder = roc_auc_score(lb.transform(y_test.reshape(y_test.shape[0], 1)), 
                                                       pipe_convolutional2D_autoencoder.predict_proba(X_test),
                                                       average="weighted")
    print(metric_convolutional2D_autoencoder)

In [ ]:
# In development.
# Some transfer learning stuff.
# Probably will come in as another repo.
autoencoder.layers[0:10]

for layer in autoencoder.layers[0:10]:
    layer.trainable = False

autoencoder.layers[9].output

classifier_output = autoencoder.layers[9].output
classifier_output = Dense(100, activation="elu")(classifier_output)
classifier_output = Dense(100, activation="elu")(classifier_output)
classifier_output = Dense(10, activation="softmax")(classifier_output)

model_final = Model(autoencoder.input, classifier_output)
model_final.compile(loss = "categorical_crossentropy", 
                    optimizer=keras.optimizers.Adam(), 
                    metrics=["accuracy"])

model_final.fit(X_train, keras.utils.to_categorical(y_train, 10),
                validation_split=0.3,
                epochs=50,
                batch_size=100,
                shuffle=True,
                callbacks=callbacks_list, 
                verbose=2)

model_final.predict(X_test)

from sklearn.metrics import roc_auc_score
roc_auc_score(lb.transform(y_test.reshape(y_test.shape[0], 1)), 
              model_final.predict(X_test), 
              average="weighted")